# AI model-1 (pH & H2O)-------

## 原始數據匯入

In [1]:
#先把mpy.csv的data匯入，使用numpy矩陣型態
import numpy as np
import pandas as pd
import time
np.set_printoptions(suppress=True)
pph = pd.read_csv('C:/Users/bigje/Desktop/predict-PH-H2O.csv',
                 index_col=0,
                 parse_dates=True,
                 encoding='latin-1')

## 數據切割：切割15%的驗證數據並儲存成.csv

In [2]:
from sklearn.model_selection import train_test_split
#random_state隨意給定，主要是讓每次抽樣都是隨機不一樣的。
#test_size只測試資料占比多少
x_train, x_test, y_train, y_test = train_test_split(pph.loc[:, ['Feed', 'H2O', 'HCL', 'LPG', 'C4']], 
                                                    pph.loc[:, ['H2O-float', 'pH']], 
                                                    test_size=0.15,
                                                    random_state = 87)
#var的Data'x & y合併(pd.concat用在欄位合併)
xy_var = pd.concat([x_test, y_test], axis=1)
# #儲存驗證集檔案
# xy_var.to_csv('C:/Users/bigje/Desktop/(py)var15%.csv')
#85%的資料 x&y合併
xy_train = pd.concat([x_train, y_train], axis=1)
# #儲存起來
# xy_train.to_csv('C:/Users/bigje/Desktop/(py)train85%.csv')

In [3]:
# #檢查分割出來的訓練與測試資料的筆數
# print(len(x_train), len(y_train), len(x_test), len(y_test))

## 再將剩餘的85%數據，切割成80/20的訓練集與測試集

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x_train, 
                                                    y_train, 
                                                    test_size=0.2,
                                                    random_state = 87)
#print(len(x_train), len(y_train), len(x_test), len(y_test))
#train合併、test合併
xy_train = pd.concat([x_train, y_train], axis=1)
xy_test = pd.concat([x_test, y_test], axis=1)
# #儲存訓練集與測試集
# xy_train.to_csv('C:/Users/bigje/Desktop/(py)train80%.csv')
# xy_test.to_csv('C:/Users/bigje/Desktop/(py)test20%.csv')

In [3]:
# #若不在重新抽樣可叫出先前抽樣的資料
# import numpy as np
# import pandas as pd
# x_train = pd.read_csv('C:/Users/bigje/Desktop/FCFC-chem1/FCFC-chem1-data-csv/(py)train80%.csv',
#                  index_col=0,
#                  parse_dates=True,
#                  encoding='latin-1')
# y_train = x_train.loc[:, ['H2O-float', 'pH']]
# x_train = x_train.loc[:,['Feed', 'H2O', 'HCL', 'LPG', 'C4']]

# x_test = pd.read_csv('C:/Users/bigje/Desktop/FCFC-chem1/FCFC-chem1-data-csv/(py)test20%.csv',
#                  index_col=0,
#                  parse_dates=True,
#                  encoding='latin-1')
# y_test = x_test.loc[:, ['H2O-float', 'pH']]
# x_test = x_test.loc[:, ['Feed', 'H2O', 'HCL', 'LPG', 'C4']]

### 轉成np格式，於Keras NN中需用矩陣型態

In [4]:
npx_train = x_train.to_numpy() #x轉成np格式
npy_train = y_train.to_numpy() #y轉乘np格式
npx_test = x_test.to_numpy()
npy_test = y_test.to_numpy()
type(npx_train)
# print(npx_train)
# print(npy_train)

numpy.ndarray

### 將np下的自變數與依變數存成pkl，以便還原正規化後的預測值使用

In [5]:
import joblib
joblib.dump(npx_train, 'C:/Users/bigje/Desktop/FCFC-chem1/Corrosion_rate_model/minmax-x.pkl')
joblib.dump(npy_train, 'C:/Users/bigje/Desktop/FCFC-chem1/Corrosion_rate_model/minmax-y.pkl')

['C:/Users/bigje/Desktop/minmax-y.pkl']

### 將數值都正規化(min-max)(0~1之間)

In [6]:
from sklearn import preprocessing
#訓練與測試的x&y正規化需分開設定
trainx = preprocessing.MinMaxScaler()
trainy = preprocessing.MinMaxScaler()
testx = preprocessing.MinMaxScaler()
testy = preprocessing.MinMaxScaler()

#fit_transform()為正規化(0~1之間)
trainx_minmax = trainx.fit_transform(npx_train).reshape(npx_train.shape[0],npx_train.shape[1])
trainy_minmax = trainy.fit_transform(npy_train).reshape(npy_train.shape[0],npy_train.shape[1])
testx_minmax = testx.fit_transform(npx_test).reshape(npx_test.shape[0],npx_test.shape[1])
testy_minmax = testy.fit_transform(npy_test).reshape(npy_test.shape[0],npy_test.shape[1])
# print(trainx_minmax)
# print(trainy_minmax)
#正規化的訓練集&測試集
x_training = trainx_minmax
y_training = trainy_minmax
x_testing = testx_minmax
y_testing = testy_minmax

## 先安裝Keras，需與tensorflow(2.2以上(含))版本匹配

In [7]:
#開始導入keras模型
from keras.models import Sequential
model = Sequential()

In [8]:
from keras.layers import Dense

model.add(Dense(units=128, activation='relu', input_dim=5, kernel_initializer='he_normal'))
model.add(Dense(units=128, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(units=2, activation='sigmoid'))

In [9]:
#from keras.optimizers import adam
#loss function ：L2
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [10]:
#查看Keras類神經模型有幾層跟詳細架構
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               768       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 17,538
Trainable params: 17,538
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(x_training, y_training, epochs=500, batch_size=16)

Epoch 1/500
343/343 [==============================] - 1s 3ms/step - loss: 0.0100 - accuracy: 0.8844
Epoch 2/500
343/343 [==============================] - 1s 3ms/step - loss: 0.0015 - accuracy: 0.9635
Epoch 3/500
343/343 [==============================] - 1s 3ms/step - loss: 7.2905e-04 - accuracy: 0.9757
Epoch 4/500
343/343 [==============================] - 1s 3ms/step - loss: 4.1901e-04 - accuracy: 0.9814
Epoch 5/500
343/343 [==============================] - 1s 3ms/step - loss: 2.6580e-04 - accuracy: 0.9865
Epoch 6/500
343/343 [==============================] - 1s 3ms/step - loss: 2.1864e-04 - accuracy: 0.9858
Epoch 7/500
343/343 [==============================] - 1s 3ms/step - loss: 1.6102e-04 - accuracy: 0.9876
Epoch 8/500
343/343 [==============================] - 1s 3ms/step - loss: 1.5138e-04 - accuracy: 0.9863
Epoch 9/500
343/343 [==============================] - 1s 3ms/step - loss: 1.4208e-04 - accuracy: 0.9883
Epoch 10/500
343/343 [==============================] - 1s 2ms/

343/343 [==============================] - 1s 3ms/step - loss: 9.7419e-06 - accuracy: 0.9956
Epoch 153/500
343/343 [==============================] - 1s 2ms/step - loss: 8.2516e-06 - accuracy: 0.9964
Epoch 154/500
343/343 [==============================] - 1s 2ms/step - loss: 6.1558e-06 - accuracy: 0.9969
Epoch 155/500
343/343 [==============================] - 1s 3ms/step - loss: 1.1263e-05 - accuracy: 0.9942
Epoch 156/500
343/343 [==============================] - 1s 2ms/step - loss: 1.1704e-05 - accuracy: 0.9953
Epoch 157/500
343/343 [==============================] - 1s 2ms/step - loss: 8.3674e-06 - accuracy: 0.9945
Epoch 158/500
343/343 [==============================] - 1s 2ms/step - loss: 4.8458e-06 - accuracy: 0.9960
Epoch 159/500
343/343 [==============================] - 1s 2ms/step - loss: 7.2401e-06 - accuracy: 0.9967
Epoch 160/500
343/343 [==============================] - 1s 2ms/step - loss: 1.0256e-05 - accuracy: 0.9949
Epoch 161/500
343/343 [============================

343/343 [==============================] - 1s 3ms/step - loss: 3.1160e-06 - accuracy: 0.9984
Epoch 304/500
343/343 [==============================] - 1s 3ms/step - loss: 3.4251e-06 - accuracy: 0.9969
Epoch 305/500
343/343 [==============================] - 1s 3ms/step - loss: 4.5394e-06 - accuracy: 0.9969
Epoch 306/500
343/343 [==============================] - 1s 2ms/step - loss: 8.0185e-06 - accuracy: 0.9953
Epoch 307/500
343/343 [==============================] - 1s 2ms/step - loss: 2.3650e-06 - accuracy: 0.9978
Epoch 308/500
343/343 [==============================] - 1s 3ms/step - loss: 4.7101e-06 - accuracy: 0.9971
Epoch 309/500
343/343 [==============================] - 1s 2ms/step - loss: 6.1834e-06 - accuracy: 0.9967
Epoch 310/500
343/343 [==============================] - 1s 2ms/step - loss: 3.0410e-06 - accuracy: 0.9980
Epoch 311/500
343/343 [==============================] - 1s 2ms/step - loss: 2.9944e-06 - accuracy: 0.9978
Epoch 312/500
343/343 [============================

343/343 [==============================] - 1s 2ms/step - loss: 2.3507e-06 - accuracy: 0.9969
Epoch 456/500
343/343 [==============================] - 1s 2ms/step - loss: 2.6195e-06 - accuracy: 0.9982
Epoch 457/500
343/343 [==============================] - 1s 2ms/step - loss: 2.2866e-06 - accuracy: 0.9971
Epoch 458/500
343/343 [==============================] - 1s 2ms/step - loss: 3.9864e-06 - accuracy: 0.9969
Epoch 459/500
343/343 [==============================] - 1s 2ms/step - loss: 3.8075e-06 - accuracy: 0.9971
Epoch 460/500
343/343 [==============================] - 1s 2ms/step - loss: 2.5046e-06 - accuracy: 0.9971
Epoch 461/500
343/343 [==============================] - 1s 2ms/step - loss: 4.3257e-06 - accuracy: 0.9969
Epoch 462/500
343/343 [==============================] - 1s 3ms/step - loss: 1.8057e-06 - accuracy: 0.9985
Epoch 463/500
343/343 [==============================] - 1s 2ms/step - loss: 3.4033e-06 - accuracy: 0.9978
Epoch 464/500
343/343 [============================

In [12]:
#評估模型 loss & accuracy
loss_metrics = model.evaluate(x_testing, y_testing)

43/43 [==============================] - 0s 1ms/step - loss: 7.2364e-06 - accuracy: 0.9971


In [13]:
#[round(i, 5) for i in loss_metrics]

## 用測試集進行預測

In [14]:
pred0 = model.predict(x_testing) #.reshape(npy_test.shape[0],npy_test.shape[1])
#numpy 列合併
pred1 = np.column_stack((pred0, y_testing))

## 將測試的集預測值還原

In [17]:
re_pred1 = testy.inverse_transform(pred1[:,:-2]) #前兩行(到第三欄前的都要)(預測值)
re_pred2 = testy.inverse_transform(pred1[:,2:]) #後兩行(從第三欄後的到最後都要)(實際值)
re_pred = np.column_stack((re_pred1, re_pred2))
print(re_pred)

[[ 8.23907102 -0.28172951  8.24264728 -0.28285123]
 [13.45968741  0.20934006 13.49651103  0.21025867]
 [12.05619002  0.0553987  12.07624554  0.05339629]
 ...
 [17.92706462  0.12569643 17.98507311  0.12706249]
 [ 3.332576   -0.15369529  3.45638018 -0.16179619]
 [12.18556611  0.54087814 12.15723378  0.54303634]]


## 模型評估指標

In [19]:
#RMSE
def rmse(predict, actual):
    return np.sqrt(((predict - actual) ** 2).mean())
#MAPE
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))*100
#SMAPE
def smape(actual, predict):
    return 1/len(actual) * np.sum(2 * np.abs(predict-actual) / (np.abs(actual) + np.abs(predict))*100)

from sklearn.metrics import r2_score
#r2_score(y_true, y_pred)

In [20]:
#H2O的RMSE
rmse_H2O = rmse(re_pred[:,0].reshape(re_pred.shape[0],1), re_pred[:,2].reshape(re_pred.shape[0],1))
#PH的RMSE
rmse_PH = rmse(re_pred[:,1].reshape(re_pred.shape[0],1), re_pred[:,3].reshape(re_pred.shape[0],1))

#H2O的MAPE
mape_H2O = mape(re_pred[:,2].reshape(re_pred.shape[0],1), re_pred[:,0].reshape(re_pred.shape[0],1))
#PH的MAPE
mape_PH = mape(re_pred[:,3].reshape(re_pred.shape[0],1), re_pred[:,1].reshape(re_pred.shape[0],1))

# #Y1的SMAPE
# smapeY1 = smape(pred1[:,2].reshape(pred1.shape[0],1), pred1[:,0].reshape(pred1.shape[0],1))
# #Y2的SMAPE
# smapeY2 = smape(pred1[:,3].reshape(pred1.shape[0],1), pred1[:,1].reshape(pred1.shape[0],1))

print('R^2_H2O：' ,r2_score(re_pred[:,2].reshape(re_pred.shape[0],1), re_pred[:,0].reshape(re_pred.shape[0],1)),"\n"
      'RMSE_H2O：' ,rmse_H2O,"\n"
      'MAPE_H2O：' ,mape_H2O,"%","\n"
      '---------------------------',"\n"
      'R^2_PH：',r2_score(re_pred[:,3].reshape(re_pred.shape[0],1), re_pred[:,1].reshape(re_pred.shape[0],1)), "\n"
      'RMSE_PH：' ,rmse_PH,"\n"
      'MAPE_PH：',mape_PH,"%")

# print(rmse_H2O, rmse_PH, mape_H2O, mape_PH, 
#       r2_score(re_pred[:,2].reshape(re_pred.shape[0],1), re_pred[:,0].reshape(re_pred.shape[0],1)),
#       r2_score(re_pred[:,3].reshape(re_pred.shape[0],1), re_pred[:,1].reshape(re_pred.shape[0],1)) )

R^2_H2O： 0.999682325601027 
RMSE_H2O： 0.16033728479266604 
MAPE_H2O： 0.5839023541538656 % 
--------------------------- 
R^2_PH： 0.9999519892346866 
RMSE_PH： 0.002440352311850585 
MAPE_PH： 1.732060244706372 %


## Keras模型只能使用HDF5(.h5)的儲存方式

In [30]:
model.save('C:/Users/bigje/Desktop/FCFC-chem1/Corrosion_rate_model/AI-model1.h5')

In [22]:
# #讀取.h5的格式檔案
# import tensorflow as tf
# reload_model = tf.keras.models.load_model('C:/Users/bigje/Desktop/keras_model.h5')
# #查看load下來的檔案
# reload_model.summary()

# ※※※※※※※※※※※※※※※※※※※※

# AI model-2(API 581)-------

In [23]:
#先把mpy.csv的data匯入，使用numpy矩陣型態
import numpy as np
import pandas as pd
import time
np.set_printoptions(suppress=True)
mpy1 = pd.read_csv('C:/Users/bigje/Desktop/FCFC-chem1/mpy(1).csv',
                 index_col=0,
                 parse_dates=True,
                 encoding='latin-1')

In [24]:
#mpy1
mpy2 = mpy1.to_numpy() #轉成np格式
# type(mpy2)

In [25]:
#API 581為training data
#目標欄需為整數，因為是分群(分類)問題
X_train = mpy2[:,0:2]
y_train = mpy2[:,2]*100

# sklearn's KNN

In [26]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #分2群
knn1 = knn.fit(X_train, y_train)

# 儲存成pkl

In [27]:
import joblib
joblib.dump(knn1, 'C:/Users/bigje/Desktop/FCFC-chem1/Corrosion_rate_model/AI-model2.pkl')

['C:/Users/bigje/Desktop/AI-model2.pkl']

In [44]:
# #讀取pkl
# knn2 = joblib.load('C:/Users/bigje/Desktop/AI-model2.pkl')

# ※※※※※※※※※※※※※※※※※※※※

# 新數據進行串接-------

## 5個inputs(feed,H2O,HCI,LPG,C4) 跟各管段溫度可自行輸入

In [1]:
import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
Input_data = dict({
    '3C250_feed': float(input('入料量：')),
    '3C250_H2O': float(input('含水量：')),
    '3C250_HCL': float(input('含氯量：')),
    'LPG': float(input('LPG：')),
    'C4': float(input('C4：')),
    '3C250_temp': float(input('管段1溫度：')),  #管段1  溫度
    '3E254_temp': float(input('管段2溫度：')),   #管段2  溫度
    '3E255_temp': float(input('管段3,4溫度：')),  #管段34 溫度
})
#儲存input data的數值於numpy格式
xxtest =np.array([[Input_data['3C250_feed'], Input_data['3C250_H2O'], Input_data['3C250_HCL'], Input_data['LPG'], 
                   Input_data['C4']]])
#匯入AI模型1跟2的.h5&.pkl檔
AI_model1 = tf.keras.models.load_model('C:/Users/bigje/Desktop/FCFC-chem1/Corrosion_rate_model/AI-model1.h5')
AI_model2 = joblib.load('C:/Users/bigje/Desktop/FCFC-chem1/Corrosion_rate_model/AI-model2.pkl')
#導入自變數與依變數的原檔(.pkl)
npx_train = joblib.load('C:/Users/bigje/Desktop/FCFC-chem1/Corrosion_rate_model/minmax-x.pkl')
npy_train = joblib.load('C:/Users/bigje/Desktop/FCFC-chem1/Corrosion_rate_model/minmax-y.pkl')
#------------------------------------------------------------------------------------------------------------------#
#正規化input data，以便AI model1使用
#feed
def nx1(feed):
    return  (feed - npx_train[:,0].min()) / (npx_train[:,0].max()-npx_train[:,0].min())
#H2O
def nx2(H2O):
    return  (H2O - npx_train[:,1].min()) / (npx_train[:,1].max()-npx_train[:,1].min())
#HCL
def nx3(HCL):
    return  (HCL - npx_train[:,2].min()) / (npx_train[:,2].max()-npx_train[:,2].min())
#LPG
def nx4(LPG):
    return  (LPG - npx_train[:,3].min()) / (npx_train[:,3].max()-npx_train[:,3].min())
#C4
def nx5(C4):
    return  (C4 - npx_train[:,4].min()) / (npx_train[:,4].max()-npx_train[:,4].min())
#H2O-float
def ny1(H2O_float):
    return  (H2O_float * (npy_train[:,0].max() - npy_train[:,0].min()) + npy_train[:,0].min() )
#pH
def ny2(pH):
    return  (pH * (npy_train[:,1].max() - npy_train[:,1].min()) + npy_train[:,1].min() )
#整理全部的輸入變數
n1 = nx1(xxtest[:,0])
n2 = nx2(xxtest[:,1])
n3 = nx3(xxtest[:,2])
n4 = nx4(xxtest[:,3])
n5 = nx5(xxtest[:,4])
#合併
xxtesting = np.column_stack((n1,n2,n3,n4,n5))

#進入AI model1預測
xxpred0 = AI_model1.predict(xxtesting) #.reshape(npy_test.shape[0],npy_test.shape[1])
#預測值還原
xxy1 = ny1(xxpred0[:,0])
xxy2 = ny2(xxpred0[:,1])
re_pred0 = np.column_stack((xxy1,xxy2))
#------------------------------------------------------------------------------------------------------------------#
#操作溫度與pH預測值合併
ne1 = np.c_[Input_data['3C250_temp'],re_pred0[:,1]]
ne2 = np.c_[Input_data['3E254_temp'],re_pred0[:,1]]
ne34 = np.c_[Input_data['3E255_temp'],re_pred0[:,1]]

#進入AI model2預測腐蝕率(beta)
pred1 = AI_model2.predict(ne1)
pred2 = AI_model2.predict(ne2)
pred34 = AI_model2. predict(ne34)
beta1 = pred1/100
beta2 = pred2/100
beta34 = pred34/100
#------------------------------------------------------------------------------------------------------------------#
import pandas as pd
import numpy as np
su1 = pd.read_excel('C:/Users/bigje/Desktop/FCFC-chem1/修正係數/新修正係數.xlsx', skiprows = 2, sheet_name = None)
su = su1.get('Corrosion')     #修正係數表單區
rate = su1.get('Interaction') #層厚影響的佔水比例表單
suw = su.columns[6]           #水膜厚度
su.columns = ['X','Section','x1','x2','x3','x4','x5','x6','x7','x8','alpha']

rate1 = rate.to_numpy()
rate1 = rate1[8:12,8]        #層厚影響的佔水比例
su = su.to_numpy()
su_x4_sum = su[1:5,5].sum()  #各管段內部容積*各管段真實密度之總和

#各管段的修正係數公式(輸入H2O預測值，x1)
def section1(x1):
    return ((((x1/su_x4_sum*su[1,5])*rate1[0])*1000/suw/10000)/su[1,2])*2
def section2(x1):
    return ((((x1/su_x4_sum*su[2,5])*rate1[1])*1000/suw/10000)/su[2,2])*2
def section3(x1):
    return ((((x1/su_x4_sum*su[3,5])*rate1[2])*1000/suw/10000)/su[3,2])*2
def section4(x1):
    return ((((x1/su_x4_sum*su[4,5])*rate1[3])*1000/suw/10000)/su[4,2])*2
#各管段修正係數(alpha)
s1alpha = np.around(section1(re_pred0[:,0]),5)
s2alpha = np.around(section2(re_pred0[:,0]),5)
s3alpha = np.around(section3(re_pred0[:,0]),5)
s4alpha = np.around(section4(re_pred0[:,0]),5)
# print(f's1alpha：{s1alpha}\ns2alpha：{s2alpha}\ns3alpha：{s3alpha}\ns4alpha：{s4alpha}')
#------------------------------------------------------------------------------------------------------------------#
#修正後腐蝕率
CR1 = beta1 * s1alpha
CR2 = beta2 * s2alpha
CR3 = beta34 * s3alpha
CR4 = beta34 * s4alpha
#------------------------------------------------------------------------------------------------------------------#
#將所有輸出結果存在字典裡
model_pred = dict({
    '3V256_3C250': dict({
        'PH': re_pred0[:,1],
        'H2O': re_pred0[:,0],
        'alpha': s1alpha,
        'corrosion': beta1,
        'correction_corrosion': CR1,
    }),
    '3C250_3E254': dict({
        'PH': re_pred0[:,1],
        'H2O': re_pred0[:,0],
        'alpha': s2alpha,
        'corrosion': beta2,
        'correction_corrosion': CR2,
    }),
    '3E254_3E255': dict({
        'PH': re_pred0[:,1],
        'H2O': re_pred0[:,0],
        'alpha': s3alpha,
        'corrosion': beta34,
        'correction_corrosion': CR3,
    }),
    '3E255_2V256': dict({
        'PH': re_pred0[:,1],
        'H2O': re_pred0[:,0],
        'alpha': s4alpha,
        'corrosion': beta34,
        'correction_corrosion': CR4,
    }),
    
})
print(f"-----\n管段1-PH：{model_pred['3V256_3C250']['PH']}\n管段1-H2O：{model_pred['3V256_3C250']['H2O']}\n管段1-alpha：{model_pred['3V256_3C250']['alpha']}\n管段1-corrosion：{model_pred['3V256_3C250']['corrosion']}\n管段1-correction_corrosion：{model_pred['3V256_3C250']['correction_corrosion']}\n-----")
print(f"管段2-PH：{model_pred['3C250_3E254']['PH']}\n管段2-H2O：{model_pred['3C250_3E254']['H2O']}\n管段2-alpha：{model_pred['3C250_3E254']['alpha']}\n管段2-corrosion：{model_pred['3C250_3E254']['corrosion']}\n管段2-correction_corrosion：{model_pred['3C250_3E254']['correction_corrosion']}\n-----")
print(f"管段3-PH：{model_pred['3E254_3E255']['PH']}\n管段3-H2O：{model_pred['3E254_3E255']['H2O']}\n管段3-alpha：{model_pred['3E254_3E255']['alpha']}\n管段3-corrosion：{model_pred['3E254_3E255']['corrosion']}\n管段3-correction_corrosion：{model_pred['3E254_3E255']['correction_corrosion']}\n-----")
print(f"管段4-PH：{model_pred['3E255_2V256']['PH']}\n管段4-H2O：{model_pred['3E255_2V256']['H2O']}\n管段4-alpha：{model_pred['3E255_2V256']['alpha']}\n管段4-corrosion：{model_pred['3E255_2V256']['corrosion']}\n管段4-correction_corrosion：{model_pred['3E255_2V256']['correction_corrosion']}\n-----")

入料量：170
含水量：50
含氯量：1
LPG：0.001
C4：0.001
管段1溫度：68.2
管段2溫度：55
管段3,4溫度：44.3
-----
管段1-PH：[-0.5026866]
管段1-H2O：[14.915763]
管段1-alpha：[0.01031]
管段1-corrosion：[25.37]
管段1-correction_corrosion：[0.2615647]
-----
管段2-PH：[-0.5026866]
管段2-H2O：[14.915763]
管段2-alpha：[0.01304]
管段2-corrosion：[25.37]
管段2-correction_corrosion：[0.33082479]
-----
管段3-PH：[-0.5026866]
管段3-H2O：[14.915763]
管段3-alpha：[0.0112]
管段3-corrosion：[25.37]
管段3-correction_corrosion：[0.28414399]
-----
管段4-PH：[-0.5026866]
管段4-H2O：[14.915763]
管段4-alpha：[0.01165]
管段4-corrosion：[25.37]
管段4-correction_corrosion：[0.29556049]
-----
